In [54]:
import pandas as pd
import numpy as np
import re
import json
import time

In [27]:
data = 'cleaned_history_.json'
unprocessed_data = pd.read_json(data)

In [28]:
unprocessed_data.head(5)

,favicon_url,page_transition,title,url,client_id,time_usec,link
0,https://www.vg.no/gfx/icons/apple-touch-icon-1...,TYPED,Nyheter fra Norges mest leste nettavis – VG,https://www.vg.no/,Iiteen/E0kB7QdKeRu1j8Q==,1665346443057756,www.vg.no
1,None,RELOAD,Ny oppskyting fra Kim - USA rådfører seg med a...,https://www.vg.no/nyheter/utenriks/i/kE6V7X/ny...,Iiteen/E0kB7QdKeRu1j8Q==,1665312300407971,www.vg.no
2,https://www.vg.no/cnp-assets/favicon-29bc799f/...,LINK,Zelenskyj med kryptisk Krim-melding – VG,https://www.vg.no/nyheter/utenriks/i/O8rdRw/ze...,Iiteen/E0kB7QdKeRu1j8Q==,1665312249062191,www.vg.no
3,https://www.vg.no/cnp-assets/favicon-29bc799f/...,LINK,Zelenskyj tar Japans parti i «krig» med Russland…,https://www.vg.no/nyheter/utenriks/i/696RlL/ze...,Iiteen/E0kB7QdKeRu1j8Q==,1665312169648724,www.vg.no
4,https://www.vg.no/cnp-assets/favicon-29bc799f/...,LINK,Haaland hylles verden rundt: – Det er brutalt ...,https://www.vg.no/sport/fotball/i/EQae6G/haala...,Iiteen/E0kB7QdKeRu1j8Q==,1665312065516677,www.vg.no


In [29]:
unprocessed_data = unprocessed_data.reset_index()
unprocessed_data = unprocessed_data.rename(columns={"index":"id"})

In [45]:
unprocessed_data
data = [(src, dst) for src, dst in zip(unprocessed_data['id'], unprocessed_data['time_usec'])]

In [96]:
def Sort(sub_li):
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of 
    # sublist lambda has been used
    return(sorted(sub_li, key = lambda x: x[1]))    
  
# Driver Code
print(Sort(data))
data = np.array(Sort(data))

[(149, 1661175088282028), (148, 1661175091402120), (147, 1661175113250506), (146, 1661175188508216), (145, 1661234269630868), (144, 1661234269635491), (143, 1661240414177422), (142, 1661240418438731), (141, 1661243244271721), (140, 1661243244276620), (139, 1661247955097877), (308, 1661247975047118), (309, 1661247975047118), (306, 1661247985480733), (307, 1661247985480733), (304, 1661248076078824), (305, 1661248076078824), (302, 1661248160037859), (303, 1661248160037859), (300, 1661348924082368), (301, 1661348924082368), (138, 1661348924087241), (298, 1661348924087375), (299, 1661348924087375), (322, 1661348929445939), (321, 1661351407094192), (320, 1661351412811720), (319, 1661351415334184), (318, 1661351519563767), (317, 1661355855341375), (316, 1661355855345947), (315, 1661355855346014), (314, 1661355858214755), (313, 1661355858214807), (312, 1661509911022686), (311, 1661509911022740), (296, 1661509924923843), (297, 1661509924923843), (294, 1661509932513894), (295, 1661509932513894),

In [94]:
from datetime import datetime
import time

def show_time(stamp):
    times = stamp / 1000000
    return datetime.utcfromtimestamp(times).strftime("%D %H:%M")
times = 1665312603834859/1000000
datetime.utcfromtimestamp(times).strftime("%D %H:%M")

show_time(1665312603834859-(5*60000000))

'10/09/22 10:45'

In [134]:
# Bins containing lists of data, add to list if within 5 mins of last entry

class SessionString:
    def __init__(self, item_id: int, usec: int, timespan = 5):
        self.id = item_id
        self.usec = usec
        self.timespan = timespan

        self.top = self.timespan * 60000000
        self.bottom = -(self.timespan * 60000000)

        # data sctructure points
        self.before = None
        self.after = None
        

    # This will always be called on head, which is the first object timewise
    def add(self, new_entry: object):
        difference = self.usec - new_entry.usec
        if difference < self.top and difference > self.bottom:
            print(show_time(a))
            print(show_time(b))
            print(f'Within {self.timespan} mins!')
            self.place(new_entry, difference)
            return True
        else:
            print(show_time(a))
            print(show_time(b))
            print('Not Within mins!', difference / 60000000, 'minutes apart.')
            return False


    def place(self, new_entry: object, difference: int):
        head = self
        # if difference is less than 0, set new head.
        if difference < 0:
            # add 
            head.before = new_entry
            new_entry.after = head
            # add difference to bottom (maximum allowed time)
            new_entry.bottom = head.bottom - difference
            self = new_entry

        # if difference is positive, we will not replace head, but we will add to tail.
        else:
            item = head
            before = None
            # go to after untill either item is null (we reached tail)
            # or fix after.
            while item.after:

                if item.usec < new_entry.usec:
                    # if we're at tail:
                    item = item.after
                
                else: 
                    # place
                    before = item.before

                    new_entry.before = before
                    before.after = new_entry
                    new_entry.after = item
                    item.before = new_entry
                    break
            
            # if while loop ran out due to no after:
            if not item.after:
                head.bottom += difference
                item.after = new_entry
                new_entry.before = item

    def as_list(self):
        values = []
        # go to first item
        head = self.get_head()
        
        # iterate and grab all entries
        while head.after:
            values.append((head.id, head.usec))
            head = head.after

        return values
        
    def get_head(self):
        head = self
        while head.before:
            head = head.before
        return head


In [135]:
sessions = []
for entry in data:
    i = 0
    # cold start
    if len(sessions) == 0:
        entry = SessionString(item_id=entry[0], usec=entry[1])
        sessions.append([entry])
    else:
        entry = SessionString(item_id=entry[0], usec=entry[1])
        print(sessions)
        head = sessions[i][0]
        while not head.add(entry):
            i += 1

[[<__main__.SessionString object at 0x16988f0d0>]]


AttributeError: 'list' object has no attribute 'add'

In [101]:
sessions = []
len(sessions)

1

In [124]:
a = 1664623909207491
b = 1664623909207491
print(show_time(a))
print(show_time(b))
difference = a - b
if difference < (5 * 60000000) and difference > -(5 * 60000000):
    print(show_time(a))
    print(show_time(b))
    print('Within 5 mins!')
else:
    print(show_time(a))
    print(show_time(b))
    print('Not Within mins!', difference / 60000000, 'minutes apart.')


10/01/22 11:31
10/01/22 11:31
10/01/22 11:31
10/01/22 11:31
Within 5 mins!
